![erd](./images/ERD-week5-exercise.png)

## Instruction

1. ใช้ project myshop เดิมจากของ WEEK 4 มาทำต่อได้เลยครับ (ใครที่หา project ของ WEEK 4 ไม่เจอแล้ว ให้กลับไปทำตามขั้นตอนใน week4_exercises.ipynb)
2. ใช้ database `shop` เดิมได้เลยเช่นกันครับ

**หมายเหตุ: ถ้านักศึกษาใช้ database เดิมจาก WEEK4 ซึ่งนักศึกษาเคยเพิ่มหรือแก้ไขข้อมูลบางส่วนแล้ว ดังนั้น output อาจจะไม่ตรงกับในตัวอย่างทั้งหมดไม่ต้องตกใจนะครับ**

In [23]:
import os
import json
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = "true"

# import modules
from shop.models import *

In [104]:
from django.db.models import Count, F, Value, Avg, Min, Max, OuterRef, Subquery
from django.db.models.functions import Length, Upper, Concat
from django.db.models.lookups import GreaterThan

# อันนี้ลองเฉย ๆ อยากเห็นหน้าตา
from django.forms.models import model_to_dict

### 1. annotate(), F()

1.1 ให้นักศึกษา query ข้อมูล `Payment` โดยให้เพิ่ม field ราคาที่ลบกับส่วนลดแล้ว โดยกำหนดให้ชื่อ field ว่า "after_discount_price" โดยใช้แสดงข้อมูล 10 ตัวแรกเรียงตาม "after_discount_price" จากมากไปน้อย (0.25 คะแนน)

**หมายเหตุ: จะต้องใช้ annotate() นะครับ ให้เอา `Payment.price` - `Payment.discount`**

ตัวอย่าง Output

```
ID: 92, PRICE: 1200500.00, DISCOUNT 29433.25, AFTER_DISCOUNT 1171066.75
ID: 82, PRICE: 1200280.00, DISCOUNT 46229.40, AFTER_DISCOUNT 1154050.60
ID: 137, PRICE: 1200690.00, DISCOUNT 71407.25, AFTER_DISCOUNT 1129282.75
ID: 105, PRICE: 1200390.00, DISCOUNT 105019.11, AFTER_DISCOUNT 1095370.89
ID: 45, PRICE: 1218900.00, DISCOUNT 126859.95, AFTER_DISCOUNT 1092040.05
ID: 7, PRICE: 1201200.00, DISCOUNT 113446.20, AFTER_DISCOUNT 1087753.80
ID: 18, PRICE: 1202190.00, DISCOUNT 121922.64, AFTER_DISCOUNT 1080267.36
ID: 77, PRICE: 379000.00, DISCOUNT 19397.00, AFTER_DISCOUNT 359603.00
ID: 127, PRICE: 320450.00, DISCOUNT 14578.90, AFTER_DISCOUNT 305871.10
ID: 125, PRICE: 320399.00, DISCOUNT 17939.55, AFTER_DISCOUNT 302459.45
```

In [14]:
# code here
paids=Payment.objects.all().annotate(
    after_discount_price = F("price")-F("discount")
)

for pay in paids:
    print(f"ID: {pay.id}, PRICE: {pay.price}, DISCOUNT {pay.discount}\
    AFTER_DISCOUNT {pay.after_discount_price}")

ID: 62, PRICE: 600.00, DISCOUNT 56.70    AFTER_DISCOUNT 543.30
ID: 3, PRICE: 700.00, DISCOUNT 46.55    AFTER_DISCOUNT 653.45
ID: 4, PRICE: 9350.00, DISCOUNT 481.78    AFTER_DISCOUNT 8868.22
ID: 5, PRICE: 590.00, DISCOUNT 10.29    AFTER_DISCOUNT 579.71
ID: 6, PRICE: 4300.00, DISCOUNT 195.65    AFTER_DISCOUNT 4104.35
ID: 7, PRICE: 1201200.00, DISCOUNT 113446.20    AFTER_DISCOUNT 1087753.80
ID: 8, PRICE: 21900.00, DISCOUNT 2002.35    AFTER_DISCOUNT 19897.65
ID: 9, PRICE: 599.00, DISCOUNT 23.09    AFTER_DISCOUNT 575.91
ID: 10, PRICE: 2890.00, DISCOUNT 67.73    AFTER_DISCOUNT 2822.27
ID: 11, PRICE: 6499.00, DISCOUNT 468.96    AFTER_DISCOUNT 6030.04
ID: 12, PRICE: 3499.00, DISCOUNT 127.70    AFTER_DISCOUNT 3371.30
ID: 13, PRICE: 470.00, DISCOUNT 21.46    AFTER_DISCOUNT 448.54
ID: 14, PRICE: 4480.00, DISCOUNT 333.31    AFTER_DISCOUNT 4146.69
ID: 15, PRICE: 6290.00, DISCOUNT 288.93    AFTER_DISCOUNT 6001.07
ID: 16, PRICE: 5390.00, DISCOUNT 483.67    AFTER_DISCOUNT 4906.33
ID: 17, PRICE: 699.00

1.2 ต่อเนื่องจากข้อ 1.1 ให้ filter เฉพาะข้อมูล `Payment` ที่มี "after_discount_price" มากกว่า 500,000 (0.25 คะแนน)

ตัวอย่าง Output

```
ID: 92, PRICE: 1200500.00, DISCOUNT 29433.25, AFTER_DISCOUNT 1171066.75
ID: 82, PRICE: 1200280.00, DISCOUNT 46229.40, AFTER_DISCOUNT 1154050.60
ID: 137, PRICE: 1200690.00, DISCOUNT 71407.25, AFTER_DISCOUNT 1129282.75
ID: 105, PRICE: 1200390.00, DISCOUNT 105019.11, AFTER_DISCOUNT 1095370.89
ID: 45, PRICE: 1218900.00, DISCOUNT 126859.95, AFTER_DISCOUNT 1092040.05
ID: 7, PRICE: 1201200.00, DISCOUNT 113446.20, AFTER_DISCOUNT 1087753.80
ID: 18, PRICE: 1202190.00, DISCOUNT 121922.64, AFTER_DISCOUNT 1080267.36
```

In [15]:
# code here
paids=Payment.objects.annotate(
    after_discount_price = F("price")-F("discount")
).filter(after_discount_price__gt = 500000)

for pay in paids:
    print(f"ID: {pay.id}, PRICE: {pay.price}, DISCOUNT {pay.discount}\
    AFTER_DISCOUNT {pay.after_discount_price}")

ID: 7, PRICE: 1201200.00, DISCOUNT 113446.20    AFTER_DISCOUNT 1087753.80
ID: 18, PRICE: 1202190.00, DISCOUNT 121922.64    AFTER_DISCOUNT 1080267.36
ID: 45, PRICE: 1218900.00, DISCOUNT 126859.95    AFTER_DISCOUNT 1092040.05
ID: 82, PRICE: 1200280.00, DISCOUNT 46229.40    AFTER_DISCOUNT 1154050.60
ID: 92, PRICE: 1200500.00, DISCOUNT 29433.25    AFTER_DISCOUNT 1171066.75
ID: 105, PRICE: 1200390.00, DISCOUNT 105019.11    AFTER_DISCOUNT 1095370.89
ID: 137, PRICE: 1200690.00, DISCOUNT 71407.25    AFTER_DISCOUNT 1129282.75


1.3 ให้นักศึกษาเรียงลำดับข้อมูลลูกค้า (`Customer`) โดยเรียงลำดับตามลำดับตัวอักษร `น้อยไปมาก` (A-Z) จากชื่อเต็มของลูกค้า (`full_name`) โดยแสดง 5 คนแรก (0.5 คะแนน)

**Hint:** Field `full_name` นั้นจะต้องถูก annotate ขึ้นมาโดยการนำ `first_name` มาต่อกับ `last_name` โดยใช้ `Concat(*expressions, **extra)` 

**Hint:** แปลง object เป็น dict ใช้ `values()` [doc](https://docs.djangoproject.com/en/5.0/ref/models/querysets/#values)

```python
>>> Blog.objects.filter(name__startswith="Beatles").values()
<QuerySet [{'id': 1, 'name': 'Beatles Blog', 'tagline': 'All the latest Beatles news.'}]>
```

**Hint:** อยาก print dictionary สวยๆ ใช้ `json.dumps`

```python
print(json.dumps(dictionary, indent=4, sort_keys=False))
```

[doc](https://docs.djangoproject.com/en/5.0/ref/models/database-functions/#concat)

ตัวอย่าง Output 

```python
{
    "id": 17,
    "email": "anantaya.deena@gmail.com",
    "address": {
        "district": "Yan Nawa",
        "location": "60 Thanon Chan Road",
        "province": "Bangkok",
        "postal_code": 10120
    },
    "full_name": "Anantaya Tontong"
}
{
    "id": 25,
    "email": "bancha.zeyou@gmail.com",
    "address": {
        "district": "Watthana",
        "location": "6 Thong Lo Road",
        "province": "Bangkok",
        "postal_code": 10110
    },
    "full_name": "Bancha Kittisompong"
}
{
    "id": 19,
    "email": "chayapol.231@gmail.com",
    "address": {
        "district": "Hang Chat",
        "location": "160 Lampang Road",
        "province": "Lampang",
        "postal_code": 52190
    },
    "full_name": "Chayapol Komprach"
}
{
    "id": 4,
    "email": "dejwit.tt@gmail.com",
    "address": {
        "district": "Chiang Khan",
        "location": "150 Loei Road",
        "province": "Loei",
        "postal_code": 42110
    },
    "full_name": "Dejwit Tangjareonsakul"
}
{
    "id": 11,
    "email": "jack.maa@gmail.com",
    "address": {
        "district": "Bang Khen",
        "location": "88 Phahonyothin Road",
        "province": "Bangkok",
        "postal_code": 10220
    },
    "full_name": "Jack Maa"
}

```

In [166]:
# code here
customer = Customer.objects.annotate(
    full_name = Concat("first_name", Value(' '), "last_name")
).order_by("full_name")

# customer.values()
print(json.dumps(list(customer.values()), indent=4, sort_keys=False))

# print(model_to_dict(customer))



[
    {
        "id": 17,
        "first_name": "Anantaya",
        "last_name": "Tontong",
        "email": "anantaya.deena@gmail.com",
        "address": {
            "district": "Yan Nawa",
            "location": "60 Thanon Chan Road",
            "province": "Bangkok",
            "postal_code": 10120
        },
        "full_name": "Anantaya Tontong"
    },
    {
        "id": 25,
        "first_name": "Bancha",
        "last_name": "Kittisompong",
        "email": "bancha.zeyou@gmail.com",
        "address": {
            "district": "Watthana",
            "location": "6 Thong Lo Road",
            "province": "Bangkok",
            "postal_code": 10110
        },
        "full_name": "Bancha Kittisompong"
    },
    {
        "id": 19,
        "first_name": "Chayapol",
        "last_name": "Komprach",
        "email": "chayapol.231@gmail.com",
        "address": {
            "district": "Hang Chat",
            "location": "160 Lampang Road",
            "province": "Lampang

### 3. aggregation - count(), sum(), avg()


3.1 ให้นักศึกษาหาค่าเฉลี่ยของราคาสินค้า (`Product.price`) ที่มีจำนวนคงเหลือ (`Product.remaining_amount`) มากกว่า 0 (0.2 คะแนน)

``` PYTHON
{'avg': Decimal('29308.000000000000')}
```


In [48]:
# code here
Product.objects.filter(remaining_amount__gt=0).aggregate(
    avg=Avg("price", default=0)
)

# Product.objects.filter(remaining_amount__gt=0).annotate(avg=Avg("price"))

{'avg': Decimal('29308.000000000000')}

3.2 จงหาผลรวมราคา (`CartItem.product.price`) ที่อยู่ในตระกร้าสินค้าของเดือน `พฤษภาคม` (ดูจาก `Cart.create_date`) (0.2 คะแนน)

```PYTHON
{'sum': Decimal('9912555.00')}

```

In [56]:
cart = CartItem.objects.filter(cart__create_date__month=5).aggregate(
    sum=Sum("product__price", default=0)
)

cart

{'sum': Decimal('9912555.00')}

3.3 ให้นักศึกษานับจำนวนสินค้าที่อยู่ประเภท `Electronics`,  `Jewelry` และ ราคาของสินค้าอยู่ในช่วง 8,000.00 - 50,000.00 (0.2 คะแนน)

```
PRODUCT CATEGORY NAME: Electronics, PRODUCT COUNT: 6
PRODUCT CATEGORY NAME: Jewelry, PRODUCT COUNT: 1
```

In [100]:
# code here
pc = Product.objects.filter(categories__name='Electronics').aggregate(product_count=Count("name", filter=Q(price__gt=8000, price__lt=50000)))
pc2 = Product.objects.filter(categories__name__in=['Electronics', 'Jewelry']).aggregate(product_count=Count("name", filter=Q(price__gt=8000, price__lt=50000)))


print("PRODUCT CATEGORY NAME: Electronics, PRODUCT COUNT", pc['product_count'])
print("PRODUCT CATEGORY NAME: Jewelry, PRODUCT COUNT", pc2['product_count'])


PRODUCT CATEGORY NAME: Electronics, PRODUCT COUNT 6
PRODUCT CATEGORY NAME: Jewelry, PRODUCT COUNT 1


In [118]:
# จะ group อะไรก็เอาขึ้นต้น
cats = ProductCategory.objects.annotate(
    product_count=Count("product", filter=Q(product__price__range=(8000, 50000)))
).filter(name__in=("Electronics", "Jewelry"))

for cat in cats:
    print(f"PRODUCT CATEGORY NAME: {cat.name}, PRODUCT COUNT: {cat.product_count}")

PRODUCT CATEGORY NAME: Electronics, PRODUCT COUNT: 6
PRODUCT CATEGORY NAME: Jewelry, PRODUCT COUNT: 1


3.4 ให้นักศึกษามีการใช้ `Subquery` เพื่อหาคำสั่งซื้อ (`Order`) ล่าสุดของลูกค้า (`Customer`) แต่ละคน โดยมีการเรียงลำดับคำสั่งซื้อตามวันที่ที่สั่งซื้อ และแสดงผลข้อมูลตามตัวอย่างที่กำหนดไว้ (0.4 คะแนน)
```txt
# ตัวอย่างผลลัพธ์บางส่วน
CUSTOMER NAME: Sek Loso, ORDER DATE: 2024-05-13
CUSTOMER NAME: Warit Pititat, ORDER DATE: 2024-05-26
CUSTOMER NAME: Chayapol Komprach, ORDER DATE: 2024-05-26
CUSTOMER NAME: Kwantip Aekopas, ORDER DATE: 2024-05-28
CUSTOMER NAME: Kerikwan Laopong, ORDER DATE: 2024-05-30
CUSTOMER NAME: Pattanapoom Srisawangjan, ORDER DATE: 2024-05-30
CUSTOMER NAME: Manit Senapan, ORDER DATE: 2024-06-06
CUSTOMER NAME: Techathorn Sriwanwilai, ORDER DATE: 2024-06-07
CUSTOMER NAME: Anantaya Tontong, ORDER DATE: 2024-06-08
CUSTOMER NAME: Kittitorn Wilairat, ORDER DATE: 2024-06-11
...
```

In [117]:
# code here
o = Order.objects.filter(
    customer=OuterRef("pk")
).order_by('-order_date')

iii = Customer.objects.annotate(test=Subquery(o.values("order_date")[:1])).order_by('test')

for i in iii:
    print(f"CUSTOMER NAME: {i.first_name} {i.last_name}, ORDER DATE: {i.test}")
    


CUSTOMER NAME: Sek Loso, ORDER DATE: 2024-05-13
CUSTOMER NAME: Warit Pititat, ORDER DATE: 2024-05-26
CUSTOMER NAME: Chayapol Komprach, ORDER DATE: 2024-05-26
CUSTOMER NAME: Kwantip Aekopas, ORDER DATE: 2024-05-28
CUSTOMER NAME: Kerikwan Laopong, ORDER DATE: 2024-05-30
CUSTOMER NAME: Pattanapoom Srisawangjan, ORDER DATE: 2024-05-30
CUSTOMER NAME: Manit Senapan, ORDER DATE: 2024-06-06
CUSTOMER NAME: Techathorn Sriwanwilai, ORDER DATE: 2024-06-07
CUSTOMER NAME: Anantaya Tontong, ORDER DATE: 2024-06-08
CUSTOMER NAME: Kittitorn Wilairat, ORDER DATE: 2024-06-11
CUSTOMER NAME: Porntipa Pasakul, ORDER DATE: 2024-06-12
CUSTOMER NAME: Peeraya Wichienchan, ORDER DATE: 2024-06-13
CUSTOMER NAME: Rachanon Hongpatra, ORDER DATE: 2024-06-14
CUSTOMER NAME: Suwichada Jarastham, ORDER DATE: 2024-06-14
CUSTOMER NAME: Pong Sawadiwong, ORDER DATE: 2024-06-15
CUSTOMER NAME: Dejwit Tangjareonsakul, ORDER DATE: 2024-06-16
CUSTOMER NAME: Bancha Kittisompong, ORDER DATE: 2024-06-17
CUSTOMER NAME: Yanisa Nantapak

### 4. one-to-one & one-to-many

4.1 จงสร้าง Order และ Payment ของลูกค้า Sek Loso จำนวน 2 รายการ โดยแต่ละ Order มีสินค้า 1 รายการ และชำระเงินด้วย Payment คนละรายการ จากนั้น เขียน query แสดงข้อมูล Order และ Payment ทั้งหมดของ Sek Loso (0.25 คะแนน)
```txt
# ตัวอย่างผลลัพธ์บางส่วน
CUSTOMER: Sek Loso, ORDER ID: 48, PRICE: 1699.00, DISCOUNT: 112.27, PRODUCT: Bird Cage
CUSTOMER: Sek Loso, ORDER ID: 59, PRICE: 1740.00, DISCOUNT: 140.81, PRODUCT: Microwave Oven
```

In [123]:
# code here
customer = Customer.objects.get(first_name='Sek', last_name='Loso')
order = Order(
    customer=customer,
    order_date="2025-12-12"
)
order.save()

pay = Payment(
    order=order,
    payment_date="2025-12-12",
    price=1699,
)

pay.save()

product = Product.objects.get(pk=1)

orderitem = OrderItem(
    order=order,
    product=product,
)

orderitem.save()

payi = PaymentItem(
    payment=pay,
    order_item = orderitem,
    price= 1699,
)

payi.save()

In [153]:
# code here
customer = Customer.objects.get(first_name='Sek', last_name='Loso')
order2 = Order(
    customer=customer,
    order_date="2026-12-12"
)
order2.save()

pay2 = Payment(
    order=order2,
    payment_date="2026-12-12",
    price=99,
)

pay2.save()

product2 = Product.objects.get(pk=2)

orderitem2 = OrderItem(
    order=order2,
    product=product2,
)

orderitem2.save()

payi2 = PaymentItem(
    payment=pay2,
    order_item = orderitem2,
    price= 99,
)

payi2.save()

In [155]:
# ans = Order.objects.filter(
#     customer__first_name="Sek", customer__last_name="Loso"
# )

# for a in ans:
#     print(a.customer.first_name, a.)

ans = OrderItem.objects.filter(
    order__customer__first_name='Sek', order__customer__last_name='Loso'
)

for a in ans:
    print(f"CUSTOMER: {a.order.customer.first_name} {a.order.customer.last_name}, \
ORDER ID: {a.order.id}, \
PRICE: {a.order.payment.price}, \
DISCOUNT: {a.order.payment.discount}, \
PRODUCT: {a.product.name}")

CUSTOMER: Sek Loso, ORDER ID: 48, PRICE: 1699.00, DISCOUNT: 112.27, PRODUCT: Bird Cage
CUSTOMER: Sek Loso, ORDER ID: 48, PRICE: 1699.00, DISCOUNT: 112.27, PRODUCT: Hot Wheels Track Builder System Race Crate
CUSTOMER: Sek Loso, ORDER ID: 59, PRICE: 1740.00, DISCOUNT: 140.81, PRODUCT: Microwave Oven
CUSTOMER: Sek Loso, ORDER ID: 59, PRICE: 1740.00, DISCOUNT: 140.81, PRODUCT: Men's Polo Shirt
CUSTOMER: Sek Loso, ORDER ID: 186, PRICE: 1699.00, DISCOUNT: 0.00, PRODUCT: Smartphone
CUSTOMER: Sek Loso, ORDER ID: 187, PRICE: 99.00, DISCOUNT: 0.00, PRODUCT: Laptop


4.2 ให้นักศึกษาดึงข้อมูล `Payment` ที่มี `PaymentMethod` ที่เป็น 'CREDIT' และมีราคาหลังหักส่วนลด (`Payment.price` - `Payment.discount`) ต่ำกว่า 500 และให้แสดงจำนวนของ `PaymentItem` ของแต่ละรายการ Payment (0.25 คะแนน)
```txt
# ตัวอย่างผลลัพธ์
PAYMENT ID: 40, AFTER_DISCOUNT: 493.06, ITEM COUNT: 2
PAYMENT ID: 13, AFTER_DISCOUNT: 448.54, ITEM COUNT: 2
```

In [165]:
# code here
p = Payment.objects.filter(
    paymentmethod__method="CREDIT"
).annotate(d=F("price")-F("discount")
).filter(d__lt=500).annotate(payitem_count=Count("paymentitem"))


for i in p:
    print(f"PAYMENT ID: {i.id}, AFTER_DISCOUNT: {i.d}")

PAYMENT ID: 40, AFTER_DISCOUNT: 493.06
PAYMENT ID: 13, AFTER_DISCOUNT: 448.54


### 5. many-to-many

5.1 ให้นักศึกษาค้นหาข้อมูลสินค้า (`Product`) ที่อยู่ในทั้งหมวดหมู่สินค้า "Information Technology" และ "Electronics" 10 รายการแรก (เรียงลำดับด้วย `Product.id`) และแสดงชื่อประเภทสินค้า (`ProductCategory`) (0.25 คะแนน)

ตัวอย่าง Output บางส่วน

```
PRODUCT ID: 1, PRODUCT NAME: Smartphone, PRODUCT CATEGORY: Information technology, Electronics
PRODUCT ID: 2, PRODUCT NAME: Laptop, PRODUCT CATEGORY: Information technology, Electronics
PRODUCT ID: 3, PRODUCT NAME: Smart TV, PRODUCT CATEGORY: Information technology, Electronics
```

In [184]:
# code here
p = Product.objects.filter(
    Q(categories__name="Information Technology")& Q(categories__name="Electronics and Toys"))

for i in p:
    ca = i.categories.all()
    print(i.id, i.name, [n.name for n in ca])


5.2 ให้นักศึกษาทำตามขั้นตอนดังนี้ (0.25 คะแนน)

1. เปลี่ยนชื่อหมวดหมู่สินค้า `Electronics` เป็น `Electronics and Toys` แทน
2. ย้ายสินค้าที่อยู่ใน `Toys and Games` ไปอยู่ในหมวดหมู่ `Electronics and Toys` และ ลบหมวดหมู่สินค้า `Toys and Games` 
3. ค้นหาว่าสินค้าที่มีประเภทสินค้าเป็น `Electronics and Toys` ทั้งหมดมีจำนวนเท่าไหร่

In [159]:
# code here
cat = ProductCategory.objects.get(name='Electronics')
cat.name = 'Electronics and Toys'
cat.save()

In [162]:
toyp = Product.objects.filter(categories__name="Toys and Games")
toypc = ProductCategory.objects.filter(name="Toys and Games").first()

for i in toyp:
    i.categories.remove(toypc)
    i.categories.add(cat)
    
toyp.delete()



(0, {})

In [163]:
Product.objects.filter(categories__name="Electronics and Toys").count()

29